In [29]:
# Library imports
import geopandas as gpd
import osmnx as ox
import networkx as nx
import momepy
import matplotlib.pyplot as plt
import folium
import pandas as pd

In [30]:
# update osmnx settings
useful_tags_ways = ox.settings.useful_tags_way + ['cycleway'] + ['bicycle'] + ['motor_vehicle'] + ['railway'] + ['tunnel']
ox.config(use_cache=True, 
          log_console=True,
          useful_tags_way=useful_tags_ways
          )

C:\Users\b8008458\AppData\Local\Temp\ipykernel_13712\3787230940.py:3: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True,


In [31]:
# read in UK road gpkg
os_open_roads = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\ltnDetection\LTN-Detection\data\oproad_gpkg_gb\Data\oproad_roads_only.gpkg")

In [32]:
# read in UK rivers gpkg
os_open_rivers = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\ltnDetection\LTN-Detection\data\oproad_gpkg_gb\Data\oprvrs_watercourse_only.gpkg")

In [69]:
# set location and get boundary
place = "Newcastle Upon Tyne, United Kingdom"
boundary = ox.geocode_to_gdf(place)
boundary = boundary.to_crs('EPSG:27700')

In [70]:
# buffer boundary to ensure clips include riverlines which may act as borders between geographies
boundary_buffered = boundary.buffer(50)

In [71]:
## get railways

# for unknown reasons, using rail = ox.graph_from_place(place, custom_filter='["railway"]')
# doesn't ALWAYS retrive the full rail network, hence why multiple lines are used to achive the same result

# Get the major rail network
railway_types = ["","rail", "light_rail", "narrow_gauge", "subway", "tram"]

# set an empty
combined_railways = nx.MultiDiGraph()

for railway_type in railway_types:
    try:
        network = ox.graph_from_place(place, simplify=False 
                                      , custom_filter=f'["railway"~"{railway_type}"]'
                                      )
    
    # handle locations where not all rail types are found
    except Exception as e:
        print(f"No railway data found for '{railway_type}'.")
        network = nx.MultiGraph()

    combined_railways = nx.compose(combined_railways, network)
    


# convert to gdf
railways = ox.graph_to_gdfs(combined_railways, nodes=False, edges=True)

# Drop any tunnels 
railways = railways.loc[railways["tunnel"] != "yes"]


# Drop any other railway types that aren't needed
railways = railways.loc[(~railways["railway"].isin(["abandoned", "razed", "disused", "funicular", "monorail", "miniature"]))]

# set railways crs
railways = railways.to_crs('EPSG:27700')

No railway data found for 'narrow_gauge'.
No railway data found for 'subway'.
No railway data found for 'tram'.


In [72]:
## clip roads, rivers and railways to boundary


# clip
os_open_roads_clip = gpd.clip(os_open_roads, boundary_buffered)
os_open_rivers_clip = gpd.clip(os_open_rivers, boundary_buffered)
railways_clip = gpd.clip(railways, boundary_buffered)

In [73]:
# Find "neighbourhood" roads
neighbourhoods = os_open_roads_clip.loc[~((os_open_roads_clip['primary_route'] == 'True') |
                        (os_open_roads_clip['trunk_road'] == 'True') |
                        (os_open_roads_clip['fictitious'] == 'True') |
                        (os_open_roads_clip['road_classification'] == 'A Road') | 
                        (os_open_roads_clip['road_classification'] == 'B Road') | 
                        (os_open_roads_clip['road_function'] == 'Restricted Local Access Road') |
                        (os_open_roads_clip['road_function'] == 'Minor Road') |
                        (os_open_roads_clip['road_function'] == 'Motorway') |
                        (os_open_roads_clip['road_function'] == 'Minor Road') 
                        )]

In [74]:
# Find "boundary" roads
boundary_roads = os_open_roads_clip.loc[((os_open_roads_clip['primary_route'] == 'True') |
                        (os_open_roads_clip['trunk_road'] == 'True') |
                        (os_open_roads_clip['fictitious'] == 'True') |
                        (os_open_roads_clip['road_classification'] == 'A Road') | 
                        (os_open_roads_clip['road_classification'] == 'B Road') | 
                        #(os_open_roads_clip['road_function'] == 'Restricted Local Access Road') |
                        (os_open_roads_clip['road_function'] == 'Minor Road') |
                        (os_open_roads_clip['road_function'] == 'Motorway') |
                        (os_open_roads_clip['road_function'] == 'Minor Road') 
                        )]

In [75]:
## buffering and dissolving functions
 
def buffer_and_dissolve(input_gdf):
    # Buffer around boundaries
    buffered_gdf = input_gdf.copy()  # Create a copy to avoid modifying the original
    buffered_gdf['geometry'] = buffered_gdf['geometry'].buffer(25)

    # Dissolve the geometries
    dissolved_geo = buffered_gdf.unary_union

    # Create a new GeoDataFrame with the dissolved geometry
    dissolved_gdf = gpd.GeoDataFrame(geometry=[dissolved_geo])

    # Set the CRS (Coordinate Reference System)
    dissolved_gdf.crs = input_gdf.crs

    return dissolved_gdf

def dissolve_gdf(input_gdf):
    # dissolve geometries
    dissolved_geo = input_gdf.unary_union
    dissolved_gdf = gpd.GeoDataFrame(geometry=[dissolved_geo])
    dissolved_gdf.crs = input_gdf.crs

    return dissolved_gdf

In [76]:
boundary_roads = buffer_and_dissolve(boundary_roads)
boundary_rivers = buffer_and_dissolve(os_open_rivers_clip)
boundary_rail = buffer_and_dissolve(railways_clip)

In [77]:
# join all boundary features
boundaries = pd.concat([boundary_rivers, boundary_roads, boundary_rail], ignore_index=True)
boundary_features = dissolve_gdf(boundaries)

In [78]:
# Use the `difference` method to perform the "Erase" operation
erased_boundary = boundary.difference(boundary_features.unary_union)

# Convert the GeoSeries to a single geometry using unary_union
erased_boundary = erased_boundary.unary_union

# Create a new GeoDataFrame with the result of "Erase" operation
erased_boundary_gdf = gpd.GeoDataFrame(geometry=[erased_boundary], crs=boundary.crs)

# Visualize the result or save it to a file
erased_boundary_gdf.explore()